# MNIST CNN Classifier

In [1]:
import time
import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split, KFold

from utils import *

## Load the data

Split training and test sets.

In [2]:
mnist = fetch_mldata( "MNIST original" )
trainX, testX, trainY, testY = train_test_split( mnist.data, mnist.target, test_size = 10000,
                                           random_state = 123 )

trainX = trainX.reshape( (len(trainX), 28, 28, 1) )
testX  = testX.reshape(  (len(testX),  28, 28, 1) )

# Tune the model

Search for a good combinaton of hyperparameters.

In [3]:
nines = [ 0.9, 0.99, 0.999, 0.9999, 0.99999 ]
sizes = [ 100 ] #[ 32, 64, 128, 256, 512, 1024 ]
paramsList = [ { "batchSize" : n, "params" : { "alpha" : a, "momentum" : b1, "bnm" : b2 } } \
               for a in np.logspace(-4,-2, 5) \
               for b1 in nines \
               for b2 in nines \
               for n in sizes ]

loVal, trHist, vlHist, bestParams = hyperparameterSearch( trainX, trainY, paramsList, 3 )

print("\n\nLowest validation loss:", loVal)
print("Best parameters found:", bestParams )

Done 1 of 125 in 995.4861559867859s Validation loss: 0.053870033
Done 2 of 125 in 922.1060721874237s Validation loss: 0.053286657
Done 3 of 125 in 887.2604229450226s Validation loss: 0.056527894
Done 4 of 125 in 935.7274622917175s Validation loss: 0.05520611
Done 5 of 125 in 980.1910088062286s Validation loss: 0.053382147
Done 6 of 125 in 679.5440185070038s Validation loss: 0.049246084
Done 7 of 125 in 677.9487946033478s Validation loss: 0.047333285
Done 8 of 125 in 650.3688807487488s Validation loss: 0.04816192
Done 9 of 125 in 687.9942095279694s Validation loss: 0.0480514
Done 10 of 125 in 716.4446084499359s Validation loss: 0.04787107
Done 11 of 125 in 688.8144669532776s Validation loss: 0.052087456
Done 12 of 125 in 691.7934799194336s Validation loss: 0.050414678
Done 13 of 125 in 680.3268465995789s Validation loss: 0.048517898
Done 14 of 125 in 650.2617049217224s Validation loss: 0.04868573
Done 15 of 125 in 679.6158871650696s Validation loss: 0.05119558
Done 16 of 125 in 596.1192

KeyboardInterrupt: 

# Plot learning curves

Plots of the training and validation loss as functions of the epoch number.

In [ ]:
fig = plt.figure( 1, (9, 7) )

plt.plot( trHist[:], label = "training" )
plt.plot( vlHist[:], label = "validation" )

plt.xlim( xmin = 0, xmax = len(trHist) - 1 )
plt.ylim( ymin = 0, ymax = max(trHist[1], vlHist[1]) + .5 )

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.legend( loc = "best" )

plt.show()

#fig.savefig( "learning-LeNet5.eps", format = 'eps', dpi = 20000, bbox_inches = 'tight' )

# Evaluate

Check the performance on the test set.

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape = (None, n), name = "X")
y = tf.placeholder(tf.int32, shape = (None), naame = "y")
trn = tf.placeholder_with_default( False, shape = (), name = "trn" )

loss, training, accuracy, lossSummary, init, saver = mnistCNN( X, y, trn, **(bestParams["params"]) )

with tf.Session() as sess:

    saver.restore( sess, "./best/mnist-best.ckpt" )
    testAcc = accuracy.eval( feed_dict = { X : testX, y : testY })

    print( "Accuracy on test set:", testAcc )